In [1]:
import json
from pydantic import BaseModel, ValidationError
from typing import Literal
from openai import OpenAI

In [3]:
# client = OpenAI(
#     api_key=OPENAI_API_KEY,
# )

In [4]:
class ResponseSchema(BaseModel):
    hasSwimmingPool: bool
    hasBalcony: bool
    hasGarage: bool
    hasParking: bool
    hasAirConditioning: bool
    hasClotheslineSpace: bool
    hasBuiltInWardrobes: bool
    hasDishwasher: bool
    hasWashingMachine: bool
    hasDryer: bool
    hasLift: bool
    isFurnished: bool
    isPetFriendly: bool
    isSmokingAllowed: bool
    isWheelchairAccessible: bool
    hasGarden: bool
    hasFireplace: bool
    hasAlarmSystem: bool
    hasCityViews: bool
    hasWaterViews: bool
    isLuxuryListing: bool
    isTemporaryContract: bool
    isLongTermContract: bool
    isStudentFriendly: bool
    energyEfficiencyRating: Literal[
        "A+++",
        "A++",
        "A+",
        "A",
        "B",
        "C",
        "D",
        "E",
        "F",
        "Unknown",
    ]

In [5]:
def extract_features(listing_text):
    system_msg = {
        "role": "system",
        "content": (
            "You are an assistant that extracts real estate listing features from text."
            "Read the text and answer the questions below."
            "Do not include any extra explanation."
        ),
    }
    user_msg = {
        "role": "user",
        "content": f"""
        Given the following real estate listing text, please determine:
        - Does it mention a pool? (hasSwimmingPool)
        - Does it mention a balcony? (hasBalcony)
        - Does it mention a garage? (hasGarage)
        - Does it mention a parking space? (hasParking)
        - Does it mention air conditioning? (hasAirConditioning)
        - Does it mention a clothesline space? (hasClotheslineSpace)
        - Does it mention built-in wardrobes? (hasBuiltInWardrobes)
        - Does it mention a dishwasher? (hasDishwasher)
        - Does it mention a washing machine? (hasWashingMachine)
        - Does it mention a dryer? (hasDryer)
        - Does it mention an lift? (hasLift)
        - Is it furnished? (isFurnished)
        - Does it allow pets? (isPetFriendly)
        - Does it allow smoking? (isSmokingAllowed)
        - Is it wheelchair accessible? (isWheelchairAccessible)
        - Does it mention a garden? (hasGarden)
        - Does it mention a fireplace? (hasFireplace)
        - Does it mention an alarm system? (hasAlarmSystem)
        - Does it have city views? (hasCityViews)
        - Does it have water views? (hasWaterViews)
        - Is it a luxury listing? (isLuxuryListing)
        - Is it a temporary (fixed) contract? (isTemporaryContract)
        - Is it a long-term contract? (isLongTermContract)
        - Is it student-friendly? (isStudentFriendly)
        - What is the energy efficiency rating? (energyEfficiencyRating)

        If you are unsure about a particular answer, you can assume it's false or unknown.

        Text:
        \"{listing_text}\"
        """,
    }

    chat_completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[system_msg, user_msg],
        response_format=ResponseSchema,
        temperature=0,
    )

    return chat_completion

In [6]:
# Example usage:
example_description = "Encantador Penthouse no coração do Chiado, recentemente renovado e com uma área bruta de 162 m², este apartamento oferece um espaço amplo e moderno, ideal para quem procura conforto e sofisticação.  A sala de 45m² com cozinha aberta de 16m² cria um ambiente acolhedor e perfeito para receber amigos e familiares.  O terraço de 12m² é um verdadeiro refúgio, proporcionando momentos de relaxamento com uma vista deslumbrante.  Com quatro quartos, dois deles em mezzanine, e duas casas de banho completas (uma com duche e outra com banheira), este imóvel é perfeito para famílias ou para quem precisa de espaço extra. Localizado no 4º andar de um edifício com elevador, o apartamento beneficia de uma excelente orientação solar (Sul e Oeste), garantindo muita luz natural durante todo o dia. Equipado com ar condicionado e aquecimento elétrico, o conforto está assegurado em todas as estações do ano. Além disso, há a possibilidade de arrendamento a curto e médio prazo: Outubro-Maio: 8.500€/mês Junho e Setembro: 10.000€/mês Julho e Agosto: 12.000€/mês."

In [7]:
completion = extract_features(example_description)

In [8]:
content = completion.choices[0].message.content

In [9]:
try:
    # Parse and validate the response content
    solution = ResponseSchema.model_validate_json(content)
except ValidationError as e:
    # Handle validation errors
    print(e.json())

In [10]:
data = json.loads(content)

In [11]:
data

{'hasSwimmingPool': False,
 'hasBalcony': False,
 'hasGarage': False,
 'hasParking': False,
 'hasAirConditioning': True,
 'hasClotheslineSpace': False,
 'hasBuiltInWardrobes': False,
 'hasDishwasher': False,
 'hasWashingMachine': False,
 'hasDryer': False,
 'hasLift': True,
 'isFurnished': False,
 'isPetFriendly': False,
 'isSmokingAllowed': False,
 'isWheelchairAccessible': False,
 'hasGarden': False,
 'hasFireplace': False,
 'hasAlarmSystem': False,
 'hasCityViews': False,
 'hasWaterViews': False,
 'isLuxuryListing': False,
 'isTemporaryContract': True,
 'isLongTermContract': False,
 'isStudentFriendly': False,
 'energyEfficiencyRating': 'Unknown'}